### Head-to-Body Coefficient

In camera images from the museum, visitors are often not fully visible. Their bodies may be partially obscured by other visitors or exhibits, which makes it challenging to identify the image pixels corresponding to their actual positions on the floor. However, more often than not, it is possible to detect their heads.

Using the bounding box for a detected head, the height of the corresponding museum visitor can be estimated. According to the [MRI-based anatomical model of the human head](https://pmc.ncbi.nlm.nih.gov/articles/PMC2828153/#:~:text=The%20general%20shape%20of%20the,ethnicity%2C%20sex%2C%20and%20age.), head size is generally proportional to body height. Leveraging this proportionality, a custom head-to-body coefficient specific to our dataset can be determined. This coefficient, $k$, is calculated using the formula:

$
k = \frac{\text{avg body height}}{\text{avg head height}} = \frac{\sum (\text{body heights}) / N_b}{\sum (\text{head heights}) / N_h}
$

Here, $N_b$ and $N_h$ represent the total numbers of body heights and head heights, respectively, derived from the training set. This coefficient enables the estimation of a person's full body height based on the dimensions of their detected head. The estimated height can then be used to project their position onto the floor.

In [ ]:
import numpy as np
import os

HEAD_CLASS_ID = 0

# Replace with the desired path
data_dir = '/Users/eakriulin/Downloads/museum_human_v9/train/labels'

def exclude_outliers(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    return [x for x in data if lower_bound <= x <= upper_bound]

def calculate_head_to_body_coef():
    head_heights = []
    body_heights = []

    for file_name in os.listdir(data_dir):
        if not file_name.endswith('.txt'):
            continue

        file_path = f'{data_dir}/{file_name}'
        with open(file_path, 'r') as file:
            for line in file.readlines():
                attrs = line.split(' ')
                class_id = int(attrs[0])
                height = float(attrs[4])

                if class_id == HEAD_CLASS_ID:
                    head_heights.append(height)
                else:
                    body_heights.append(height)

    head_heights = exclude_outliers(head_heights)
    body_heights = exclude_outliers(body_heights)

    avg_head_height = sum(head_heights) / len(head_heights)
    avg_body_height = sum(body_heights) / len(body_heights)

    k = avg_body_height / avg_head_height
    print('Head-to-Body Coefficient: ', k)

calculate_head_to_body_coef()